In [1]:
import ipywidgets as widgets
from ipywidgets import interact, interactive, VBox, HBox, Tab
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
import numpy as np
from openquake.hazardlib.geo import Point, geodetic, NodalPlane
from openquake.hazardlib.site import Site, SiteCollection
from openquake.hazardlib.source import PointSource
from openquake.hazardlib.mfd import TruncatedGRMFD
from openquake.hazardlib.scalerel import WC1994
from openquake.hazardlib.tom import PoissonTOM
from openquake.hazardlib.pmf import PMF
from openquake.hazardlib.gsim import get_available_gsims
from openquake.hazardlib.contexts import ContextMaker
from openquake.hazardlib.imt import PGA, SA, PGV
from scipy.stats import norm, linregress
from scipy import integrate
import glob
import re

# modules
def update_point(trace, points, selector):
    idx = points.point_inds[0]
    sta_id = wbut1.DB['StationID'][idx]
    eve_id = wbut1.DB['EarthquakeId'][idx]
    wbut7.value = sta_id
    wbut8.value = eve_id
    
    # update Figure1 and 2 in tab2!
    fig1_tab2.data = []
    fig1_tab2.layout.images = []
    fig2_tab2.data = []
    fig2_tab2.layout.images = []
    
    cosmosfile = '_'.join((str(eve_id), sta_id.replace('.','_'))) + '*'
    ff = "".join(("/home/hadi/Projects/AU_gm_db/Western_Australia_DB/results/DataBase/cosmos_files/",
               cosmosfile))
    
    for fi in glob.glob(ff):
        
        if (fi.split('/')[-1].split('_')[3][-1]=='N') | (fi.split('/')[-1].split('_')[3][-1]=='1'):
            
            with open(fi) as f:
                 lines = f.readlines()
                    
            tr = np.array([])
            for l in lines[48::]:
                tr = np.concatenate((tr,[float(r) for r in re.findall(r"[-+]?(?:\d*\.\d+|\d+)", l)]), axis=None)
            
            sr = 1/200. # not sure how to get SR from cosmos files!
            t = np.arange(0, len(tr)*sr, sr)
            tr_vel = integrate.cumtrapz(tr, t, initial=0)
            tr_disp = integrate.cumtrapz(tr_vel, t, initial=0)



            fig1_tab2.add_scatter(x=t,y=tr/np.max(np.abs(tr))+2, mode='lines',
                                                  name='trace_acc', line=dict(color='black',width=1))
            fig1_tab2.add_scatter(x=t,y=tr_vel/np.max(np.abs(tr_vel))+1, 
                                                      mode='lines', 
                                                      name='trace_vel', line=dict(color='black',width=1))
            fig1_tab2.add_scatter(x=t,y=tr_disp/np.max(np.abs(tr_disp)),
                                  mode='lines', name='trace_disp', line=dict(color='black',width=1))   

#             try:
#                 df = pd.read_csv(f,delimiter=" ", skiprows=48, header=None)
#                 tr = df.values.flatten()
#                 tr = tr[~np.isnan(tr)]
#                 sr = 1/200. # not sure how to get SR from cosmos files!
#                 t = np.arange(0, len(tr)*sr, sr)
#                 tr_vel = integrate.cumtrapz(tr, t, initial=0)
#                 tr_disp = integrate.cumtrapz(tr_vel, t, initial=0)

                

#                 fig1_tab2.add_scatter(x=t,y=tr/np.max(np.abs(tr))+2, mode='lines',
#                                                       name='trace_acc', line=dict(color='black',width=1))
#                 fig1_tab2.add_scatter(x=t,y=tr_vel/np.max(np.abs(tr_vel))+1, 
#                                                           mode='lines', 
#                                                           name='trace_vel', line=dict(color='black',width=1))
#                 fig1_tab2.add_scatter(x=t,y=tr_disp/np.max(np.abs(tr_disp)),
#                                       mode='lines', name='trace_disp', line=dict(color='black',width=1))
#             except:
#                 print("OHNO")
#                 pass
            
        if (fi.split('/')[-1].split('_')[3][-1]=='E') | (fi.split('/')[-1].split('_')[3][-1]=='2'):
            
            with open(fi) as f:
                 lines = f.readlines()
                    
            tr = np.array([])
            for l in lines[48::]:
                tr = np.concatenate((tr,[float(r) for r in re.findall(r"[-+]?(?:\d*\.\d+|\d+)", l)]), axis=None)
            
            sr = 1/200. # not sure how to get SR from cosmos files!
            t = np.arange(0, len(tr)*sr, sr)
            tr_vel = integrate.cumtrapz(tr, t, initial=0)
            tr_disp = integrate.cumtrapz(tr_vel, t, initial=0)

               

            fig2_tab2.add_scatter(x=t,y=tr/np.max(np.abs(tr))+2, mode='lines',
                                                  name='trace_acc', line=dict(color='black',width=1))
            fig2_tab2.add_scatter(x=t,y=tr_vel/np.max(np.abs(tr_vel))+1, 
                                                      mode='lines', 
                                                      name='trace_vel', line=dict(color='black',width=1))
            fig2_tab2.add_scatter(x=t,y=tr_disp/np.max(np.abs(tr_disp)),
                                  mode='lines', name='trace_disp', line=dict(color='black',width=1))

    
    
def plot_gmm(dum):
    
    # DataFrame to use
    # read DB
    wbut1.DB = []
    db = pd.read_csv("./db_with_gmms.csv")
    
    # select data within dictated Magnitude and Distance
    db = db[(db['HypocentralDistance'] >= wbut5.value) & (db['HypocentralDistance'] <= wbut6.value) &
            (db['EarthquakeMagnitude'] >= wbut3.value) & (db['EarthquakeMagnitude'] <= wbut4.value)]
    # select for specific station or event, if specified!
    if (wbut9.value=='Station'):
        db = db[(db['StationID']==wbut7.value)]
    elif (wbut9.value=='Event'):
        db = db[(db['EarthquakeId']==wbut8.value)]
    
    db.reset_index(drop=True, inplace=True)
    wbut1.DB = db
    # Ref Sites and Source
    
    src_loc = Point(0., 0.)
    src_depth = 5.
    
    num_sites = 600
    points = geodetic.point_at(src_loc.longitude,src_loc.latitude,0.0, np.logspace(-1,2.7,num_sites))
    sites_list = []
    for i in np.arange(num_sites):
        site = Site(Point(points[0][i], points[1][i]), vs30=760, z1pt0=30, z2pt5=0.5,
                    vs30measured=True)
        sites_list.append(site)
    sitec = SiteCollection(sites_list)
    
    
    src = PointSource(
                    source_id='Ref',
                    name='point',
                    tectonic_region_type='Active Shallow Crust',
                    mfd=TruncatedGRMFD(min_mag=3.95, max_mag=4.05, bin_width=0.1, a_val=0.01, b_val=0.98),
                    rupture_mesh_spacing=2.,
                    magnitude_scaling_relationship=WC1994(),
                    rupture_aspect_ratio=1.,
                    temporal_occurrence_model=PoissonTOM(50.),
                    upper_seismogenic_depth=0.,
                    lower_seismogenic_depth=50.,
                    location=src_loc,
                    nodal_plane_distribution=PMF([(1., NodalPlane(strike=45, dip=50, rake=0))]),
                    hypocenter_distribution=PMF([(1, src_depth)]) #depth in km
                )
    rup = [r for r in src.iter_ruptures()][0]
    #  mean and std of Ref event predicte by selected gmm
    gs = get_available_gsims()
    gmm = gs[wbut1.value]()

    param = dict(imtls={wbut2.value: []}, cache_distances=True)
    cm = ContextMaker('*', [gmm], param)
    ctxx = cm.get_ctxs([rup], sitec)
    [ctx] = cm.get_ctxs([rup], sitec) # do not know what is happing here!
    
    gms = cm.get_mean_stds(ctxx)
    gmm_mean = gms[0][0][0]
    gmm_std = gms[1][0][0]
    
    # Normalize observations to reference event!
#     df = wbut1.DB
    df = db 
    
    # prediction of the model for reference event at the location of obsevation sites!
    gmm_mean_obs = []
    rrup_obs = []
    for index, row in df.iterrows():
        sites_obs_list = []
        site_obs = Site(Point(df['StationLongitude'][index], df['StationLatitude'][index]), 
                        vs30=760, z1pt0=30, z2pt5=0.5, vs30measured=True)
        sites_obs_list.append(site_obs)
        sitec_obs = SiteCollection(sites_obs_list)
        
        src_loc_obs = Point(df['EarthquakeLongitude'][index], df['EarthquakeLatitude'][index])
        src_obs = PointSource(
                    source_id='Ref_dum',
                    name='point',
                    tectonic_region_type='Active Shallow Crust',
                    mfd=TruncatedGRMFD(min_mag=3.95, max_mag=4.05, bin_width=0.1, a_val=0.01, b_val=0.98),
                    rupture_mesh_spacing=2.,
                    magnitude_scaling_relationship=WC1994(),
                    rupture_aspect_ratio=1.,
                    temporal_occurrence_model=PoissonTOM(50.),
                    upper_seismogenic_depth=0.,
                    lower_seismogenic_depth=50.,
                    location=src_loc_obs,
                    nodal_plane_distribution=PMF([(1., NodalPlane(strike=45, dip=50, rake=0))]),
                    hypocenter_distribution=PMF([(1, src_depth)]) #depth in km
                )
        
        rup_obs = [r for r in src_obs.iter_ruptures()][0]
        ctxx_obs = cm.get_ctxs([rup_obs], sitec_obs)
        [ctx_obs] = cm.get_ctxs([rup_obs], sitec_obs) # do not know what is happing here!

        gms_obs = cm.get_mean_stds(ctxx_obs)
        gmm_mean_obs.append(gms_obs[0][0][0][0])
        rrup_obs.append(ctx_obs.rrup[0])
       
    
    
    m1 = gmm_mean_obs
    
    m2 = df['_'.join((wbut1.value,'mean',wbut2.value))]
    
   
    
    if wbut2.value[0:2]=='SA':
        per_str = wbut2.value.replace('(',')').split(')')
        per_str = "".join(('SA(', "%.3f" % float(per_str[1]),')'))
    else:
        per_str = wbut2.value
    
    if wbut2.value=='PGV':
        scale_factor = 1.0
    else:
        scale_factor = 100.    
        
    obs_norm = (np.exp(np.log(df[per_str]/scale_factor) + m1 - m2))


    
    # plot
    fig1.data = []
    fig1.layout.images = []

    txt_sta = df['StationID']
    nk = np.empty(shape=(len(obs_norm),4), dtype='object')
    nk[:,0] = np.round(rrup_obs, 2)
    nk[:,1] = np.round(obs_norm, 2)
    nk[:,2] = df['StationID']
    nk[:,3] = df['EarthquakeId']
    
    
    fig1.add_scatter(x=np.log10(ctx.rrup), y=np.log10(np.exp(gmm_mean)),
                          mode='lines', name=wbut1.value, line=dict(color='black',width=3))
    
    fig1.add_scatter(x=np.log10(ctx.rrup), y=np.log10(np.exp(gmm_mean + gmm_std)),
                          mode='lines', name=wbut1.value, line=dict(color='black',width=2,
                                                                         dash='dash'))
    fig1.add_scatter(x=np.log10(ctx.rrup), y=np.log10(np.exp(gmm_mean - gmm_std)),
                          mode='lines', name=wbut1.value, line=dict(color='black',width=2,
                                                                        dash='dash'))
    
    fig1.add_scatter(x=np.log10(rrup_obs), y=np.log10(obs_norm),
                          mode='markers', name='Observation', marker=dict(color='indianred',size=10),
                    customdata=nk, 
                    hovertemplate='<br>Distance:%{customdata[0]}<br>Level:%{customdata[1]}<br>Station:%{customdata[2]}<br>Event:%{customdata[3]}<br>')
    
    s = fig1.data[3]
    fig1.layout.hovermode = 'closest'
    s.on_click(update_point) 
    
    # compute residuals
#     norm_res = (np.log(df[per_str]/scale_factor) - df['_'.join((wbut1.value,'mean',wbut2.value))]) / df['_'.join((wbut1.value,'std',wbut2.value))]
    res = np.log(df[per_str]/scale_factor) - df['_'.join((wbut1.value,'mean',wbut2.value))]
    norm_res = res / df['_'.join((wbut1.value,'std',wbut2.value))]
    
    # plot residual histogram
    fig2.data = []
    fig2.layout.images = []
    
    norm_res = (np.log(df[per_str]/scale_factor) - df['_'.join((wbut1.value,'mean',wbut2.value))]) / df['_'.join((wbut1.value,'std',wbut2.value))]
    
    fig2.add_histogram(x = norm_res, histnorm="probability", name='Histogram')
    mu, std = norm.fit(norm_res)
    xmin, xmax = [np.min(norm_res)-1.0, np.max(norm_res)+1.0]
    x = np.linspace(xmin, xmax, 100)
    p = norm.pdf(x, mu, std)
    q = norm.pdf(x, 0., 1.)
    fig2.add_scatter(x=x, y=p,
                          mode='lines', name='Normal (fitted)', line=dict(color='black',width=3))
    fig2.add_scatter(x=x, y=q,
                          mode='lines', name='Normal (standard)', line=dict(color='red',width=3))
    
    # plot residuals VS magnitude
    fig3.data = []
    fig3.layout.images = []
   
    slope, intercept, r_value, p_value, std_err = linregress(df['EarthquakeMagnitude'],res)
    x_line = np.linspace(min(df['EarthquakeMagnitude'])-0.5, max(df['EarthquakeMagnitude'])+0.5, 100)
    line = slope*x_line+intercept
    
    fig3.add_scatter(x=df['EarthquakeMagnitude'], y=res,
                          mode='markers', name='Residual', marker=dict(color='indianred',size=10),
                    customdata=nk, 
                    hovertemplate='<br>Distance:%{customdata[0]}<br>Level:%{customdata[1]}<br>Station:%{customdata[2]}<br>Event:%{customdata[3]}<br>')
    
    fig3.add_scatter(x=x_line, y=line,
                          mode='lines', name='Trend', line=dict(color='black',width=3))
    
    # plot residuals VS distance
    fig4.data = []
    fig4.layout.images = []
   
    slope, intercept, r_value, p_value, std_err = linregress(df['HypocentralDistance'],res)
    x_line = np.linspace(min(df['HypocentralDistance'])-10, max(df['HypocentralDistance'])+10, 100)
    line = slope*x_line+intercept
    
    fig4.add_scatter(x=df['HypocentralDistance'], y=res,
                          mode='markers', name='Residual', marker=dict(color='indianred',size=10),
                    customdata=nk, 
                    hovertemplate='<br>Distance:%{customdata[0]}<br>Level:%{customdata[1]}<br>Station:%{customdata[2]}<br>Event:%{customdata[3]}<br>')
    
    fig4.add_scatter(x=x_line, y=line,
                          mode='lines', name='Trend', line=dict(color='black',width=3))

                      
                      
                      

# list available GMMs in DB 
gmm_list = pd.read_csv("./passed_gmm.csv")
gmm_list = gmm_list['passed_gmms']
gmm_list = gmm_list.to_list()
gmm_list.insert(0,"Select Ground Motion Model")

# list available periods
period_list = ['Select Period', 'SA(0.1)', 'SA(1)']

# read DB
db = pd.read_csv("./db_with_gmms.csv")


# layout tab1
wbut1 = widgets.Dropdown(
    options=gmm_list,
    value=gmm_list[0],
    description='', layout={'width': '1100px'})

wbut2 = widgets.Dropdown(
    options=period_list,
    value=period_list[0],
    description='', layout={'width': '490px'})

wbut3 = widgets.FloatSlider(
    value=min(db["EarthquakeMagnitude"]), 
    min=min(db["EarthquakeMagnitude"]), max=max(db["EarthquakeMagnitude"]), step=0.1,
    description='Min Mag',
    continuous_update=False, layout={'width': '270px'})

wbut4 = widgets.FloatSlider(
    value=max(db["EarthquakeMagnitude"]), 
    min=min(db["EarthquakeMagnitude"]), max=max(db["EarthquakeMagnitude"]), step=0.1,
    description='Max Mag',
    continuous_update=False, layout={'width': '270px'})

wbut5 = widgets.FloatSlider(
    value=min(db['HypocentralDistance']), 
    min=min(db['HypocentralDistance']), max=max(db['HypocentralDistance']), step=1.0,
    description='Min Dist',
    continuous_update=False, layout={'width': '270px'})

wbut6 = widgets.FloatSlider(
    value=max(db['HypocentralDistance']), 
    min=min(db['HypocentralDistance']), max=max(db['HypocentralDistance']), step=1.0,
    description='Max Dist',
    continuous_update=False, layout={'width': '270px'})

sta_list = list(db['StationID'].unique())
sta_list.insert(0,"Select Station")
wbut7 = widgets.Dropdown(
    options=sta_list,
    value=sta_list[0],
    description='', layout={'width': '500px'})

eve_list = list(db['EarthquakeId'].unique())
eve_list.insert(0,"Select Earthquake")
wbut8 = widgets.Dropdown(
    options=eve_list,
    value=eve_list[0],
    description='', layout={'width': '500px'})

wbut9 = widgets.Dropdown(
    options=['All', 'Station', 'Event'],
    value='All',
    description='What to plot?', layout={'width': '590px'})

plot_button = widgets.Button(description = 'Plot', button_style='success', 
                            layout={'width': '1600px'})


# wbut1.DB = db
plot_button.on_click(plot_gmm)


fig1 = go.FigureWidget(data=[])
fig1.update_layout(
    autosize=False,
    width=1100,
    height=525,
    margin=dict(
        l=10,
        r=10,
        b=10,
        t=50,
        pad=1),
    xaxis_title="Distance (km)",
    yaxis_title="Spectral Acceleration",
    paper_bgcolor="LightSteelBlue",
    showlegend=False)

fig2 = go.FigureWidget(data=[])
fig2.update_layout(
    autosize=False,
    width=500,
    height=525,
    margin=dict(
        l=10,
        r=10,
        b=10,
        t=50,
        pad=1),
    xaxis_title="Risidual (normalized)",
    yaxis_title="Probability",
    paper_bgcolor="LightSteelBlue",
    showlegend=False)

fig3 = go.FigureWidget(data=[])
fig3.update_layout(
    autosize=False,
    width=800,
    height=525,
    margin=dict(
        l=10,
        r=10,
        b=10,
        t=50,
        pad=1),
    xaxis_title="Magnitude",
    yaxis_title="Residual",
    paper_bgcolor="LightSteelBlue",
    showlegend=False)

fig4 = go.FigureWidget(data=[])
fig4.update_layout(
    autosize=False,
    width=800,
    height=525,
    margin=dict(
        l=10,
        r=10,
        b=10,
        t=50,
        pad=1),
    xaxis_title="Distance",
    yaxis_title="Residual",
    paper_bgcolor="LightSteelBlue",
    showlegend=False)


lo_tab1 = VBox([HBox([wbut1, wbut2]), HBox([wbut3,wbut4, wbut5, wbut6]), HBox([wbut7, wbut8, wbut9]), plot_button,
          HBox([fig1,fig2]), HBox([fig3,fig4])])


# layout tab2
fig1_tab2 = go.FigureWidget(data=[])
fig1_tab2.update_layout(
    autosize=False,
    width=550,
    height=550,
    margin=dict(
        l=10,
        r=10,
        b=10,
        t=50,
        pad=1),
    xaxis_title="Time (sec)",
    yaxis_title="",
    paper_bgcolor="LightSteelBlue",
    showlegend=False)

fig2_tab2 = go.FigureWidget(data=[])
fig2_tab2.update_layout(
    autosize=False,
    width=550,
    height=550,
    margin=dict(
        l=10,
        r=10,
        b=10,
        t=50,
        pad=1),
    xaxis_title="Time (sec)",
    yaxis_title="",
    paper_bgcolor="LightSteelBlue",
    showlegend=False)

lo_tab2 = HBox([fig1_tab2, fig2_tab2])


############################# layout
tab1 = lo_tab1
tab2 = lo_tab2


tab = Tab(children=[tab1,tab2],layout={'width': 'max-content'})
tab.set_title(0,'GMMs')
tab.set_title(1,'Waveforms')


display(tab)


In [63]:
# import re
# fi = '/home/hadi/Projects/AU_gm_db/Western_Australia_DB/results/DataBase/cosmos_files/20211113130552_IU_MBWA_HN1_20_20211113130555.V2'

# with open(fi) as f:
#     lines = f.readlines()
                    
# tr = np.array([])
# for l in lines[48::]:
# #     tr.append([float(r) for r in re.findall("\d+\.\d+", l)])
#     tr = np.concatenate((tr,[float(r) for r in re.findall(r"[-+]?(?:\d*\.\d+|\d+)", l)]), axis=None)

# sr = 1/200. # not sure how to get SR from cosmos files!
# t = np.arange(0, len(tr)*sr, sr)
# tr_vel = integrate.cumtrapz(tr, t, initial=0)
# tr_disp = integrate.cumtrapz(tr_vel, t, initial=0)

In [69]:
# import matplotlib
# import matplotlib.pyplot as plt
# plt.plot(tr_disp)